In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
from pyfvm.mesh  import *
from pyfvm.field import *
from pyfvm.xnum  import *
from pyfvm.integration import *
import pyfvm.modelphy.euler as euler
import pyfvm.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 100
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

In [19]:
nx = 200
meshsim  = unimesh(ncell=nx,  length=10.)

def S(x):
    return 1.-.5*np.exp(-.5*(x-5.)**2)

plt.plot(meshsim.centers(), S(meshsim.centers())) ; plt.ylim(0,1)

model = euler.nozzle(sectionlaw=S)
bcL = { 'type': 'insub',  'ptot': 1.4, 'rttot': 1. }
bcR = { 'type': 'outsub', 'p': 1. }

rhs = modeldisc.fvm(model, meshsim, muscl(vanalbada), 
      bcL=bcL, bcR=bcR)
solver = rk3ssp(meshsim, rhs)

# computation
#
nsol    = 100
endtime = 50.
cfl     = .5

finit = rhs.fdata(model.prim2cons([  1., 0., 1. ])) # rho, u, p

start = time.clock()
fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))
cputime = time.clock()-start

print "cpu time computation (",solver.nit,"it) :",cputime,"s"
print "  %.2f µs/cell/it"%(cputime*1.e6/solver.nit/meshsim.ncell)
mach_th = np.sqrt(((bcL['ptot']/bcR['p'])**(1./3.5)-1.)/.2)
print "theoretical Mach: ", mach_th


In [20]:
# Figure / Plot of final results
varname='mach'
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.set_ylabel(varname) ; ax1.set_ylim(0., 1.1*np.max(fsol[-1].phydata(varname)))
ax1.grid(linestyle='--', color='0.5')
line1, = fsol[-1].plot(varname, 'k-', axes=ax1)
ax2.set_ylabel('error') ; ax2.set_ylim(0.001, 1.)
ax2.grid(linestyle='--', color='0.5')
ttime = [ fsol[i].time for i in range(nsol+1) ]
error = [ np.sqrt(np.sum((fsol[i].phydata(varname)-mach_th)**2)/nx)/mach_th for i in range(nsol+1) ]
#print error
line2, = ax2.semilogy(ttime, error)


In [14]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    fsol[k].set_plotdata(line1, varname)
    line2.set_data(ttime[0:k], error[0:k])
    return line1, line2

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100, blit=True)
ani
#from IPython.display import HTML
#HTML(ani.to_html5_video()) # if no rcparams